In [3]:
import subprocess

def execute_command_to_file(command):
    try:
        with open("output.txt", "w") as output_file:
            process = subprocess.run(
                command, 
                shell=True, 
                stdout=output_file, 
                stderr=output_file,
                text=True
            )
            return process.returncode
    except Exception as e:
        print(e)
        return -1

In [1]:
import os



SRC_ROOT = "D:\\projects\\hse\\java\\llm-analyzer-plugin\\src\\main\\python\\benchmark\\processed\\500"
RESULTS_ROOT = "D:\\projects\\hse\\java\\llm-analyzer-plugin\\src\\main\\python\\experiments\\semgrep"
BENCHMARK_ROOT = "D:\\projects\\hse\\java\\llm-analyzer-plugin\\src\\main\\python\\benchmark"

In [4]:
command = f"docker run --rm -v {SRC_ROOT}:/opt/src -v {RESULTS_ROOT}:/opt/results semgrep/semgrep:latest semgrep --config auto --json /opt/src --output=/opt/results/result.json"
execute_command_to_file(command)

0

In [2]:
import json

with open(os.path.join(BENCHMARK_ROOT, "processed/vulnerabilities.json")) as f:
    real = json.load(f)

print(real)

{'BenchmarkTest00001.java': {'benchmark_version': '1.2', 'category': 'pathtraver', 'test_number': 1, 'vulnerability': True, 'cwe': 22}, 'BenchmarkTest00002.java': {'benchmark_version': '1.2', 'category': 'pathtraver', 'test_number': 2, 'vulnerability': True, 'cwe': 22}, 'BenchmarkTest00003.java': {'benchmark_version': '1.2', 'category': 'hash', 'test_number': 3, 'vulnerability': True, 'cwe': 328}, 'BenchmarkTest00004.java': {'benchmark_version': '1.2', 'category': 'trustbound', 'test_number': 4, 'vulnerability': True, 'cwe': 501}, 'BenchmarkTest00005.java': {'benchmark_version': '1.2', 'category': 'crypto', 'test_number': 5, 'vulnerability': True, 'cwe': 327}, 'BenchmarkTest00006.java': {'benchmark_version': '1.2', 'category': 'cmdi', 'test_number': 6, 'vulnerability': True, 'cwe': 78}, 'BenchmarkTest00007.java': {'benchmark_version': '1.2', 'category': 'cmdi', 'test_number': 7, 'vulnerability': True, 'cwe': 78}, 'BenchmarkTest00008.java': {'benchmark_version': '1.2', 'category': 'sqli

In [3]:
with open("result.json") as f:
    raw_results = json.load(f)

results = {}

for raw_res in raw_results["results"]:
    filename = raw_res["path"].split("/")[-1]
    cwes = raw_res["extra"]["metadata"]["cwe"]

    for i in range(len(cwes)):
        cwes[i] = int(cwes[i].lstrip("CWE-").split(":")[0])
    
    if filename in results:
        for cwe in cwes:
            if cwe not in results[filename]:
                results[filename].append(cwe)
    else:
        results[filename] = cwes 

In [4]:
tp = tn = fp = fn = 0
N = 500

for filename in os.listdir(SRC_ROOT):
    vulnerable = real[filename]["vulnerability"]
    real_cwe = real[filename]["cwe"]

    result_cwes = results.get(filename, [])

    if real_cwe in result_cwes:
        if vulnerable:
            tp += 1
        else:
            fp += 1
    else:
        if vulnerable:
            fn += 1
        else:
            tn += 1

print(f"TP: {tp}, FP: {fp}, TN: {tn}, FN: {fn}")

accuracy = (tp + tn) / N
recall = tp / (tp + fn)
precision = tp / (tp + fp)
F1_score = 2 * tp / (2 * tp + fp + fn)

print(f"accuracy: {accuracy}\nprecision: {precision}\nrecall: {recall}\nF1 score: {F1_score}")

TP: 203, FP: 91, TN: 149, FN: 57
accuracy: 0.704
precision: 0.6904761904761905
recall: 0.7807692307692308
F1 score: 0.7328519855595668
